In [1]:
%matplotlib inline

In [2]:
from fenics import *
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import dolfin.fem.norms as norms


# Local imports
import modules

In [3]:
def j(U, Q):
    """
    Function to evaluate the cost functional given functions u, q.
    """
    # Compute integrals with time
    I1 = 0
    I3 = 0
    
    t = 0
    dt = T/num_steps
    for i in range(num_steps + 1):
        I1_int = assemble(Q[i]*(r-U[i])*dx(mesh))*np.exp(-rho*t)
        I3_int = assemble(Q[i]*Q[i]*dx(mesh))
        
        if i == 0 or i == num_steps - 1:
            I1_int *= 0.5
            I3_int *= 0.5
        
        I1 += I1_int
        I3 += I3_int
        
        t += dt
    
    
    I1 *= dt
    I3 *= dt*alpha/2
    
    # Compute end time integral
    I2 = lambda_*assemble(U[-1]*dx(mesh))
    
    return I1 - I2 + I3

In [4]:
def j_reduced(Q):
    """Reduced cost functional."""
    U = se.solve_state_eq(u_0, Q, V, T, num_steps, params)
    return j(U, Q)

In [5]:
def dj(U, Q, P):
    """
    Function to compute gradient of cost functional.
    """
    grad = []
    
    t = 0
    dt = T/num_steps
    for i in range(num_steps + 1):
        grad.append(np.exp(-1*rho*t)*(r - U[i]) + alpha*Q[i] - U[i]*P[i])
        t += dt
    
    grad = np.asarray(grad)
    
    return grad

In [6]:
def grad_norm(grad):
    """Compute norm of gradient."""
    norm_ = 0
    dt = T/num_steps
    
    for i in range(len(grad)):
        f = project(grad[i], V)
        int_ = norms.norm(f, "l2")
        
        if i == 0 or i == len(grad) - 1:
            int_ *= 0.5
            
        norm_ += int_
    norm_ *= dt
    
    return np.sqrt(norm_)

In [7]:
def armijo(v_k, U, Q, P):
    """Compute step length based on Armijo condition."""
    alpha = 1
    rho = 0.5
    c = 0.05
    
    phi_0 = j(U, Q)
    phi_k = j_reduced(Q + alpha*v_k)
    m = -1*grad_norm(v_k)**2
    it = 0
    while phi_k >= phi_0 + alpha*c*m and it < 20:
        alpha *= rho
        phi_k = j_reduced(Q + alpha*v_k)
        it += 1
        
        print("\n" + "="*6 + " Bactracking Line Search iteration no. {} ".format(it) + "="*6 + "\n")
        
    return [Constant(alpha), phi_k]

In [8]:
def Max(a, b):
    return (a + b + abs(a-b))/Constant(2)


def Min(a, b):
    return (a + b - abs(a-b))/Constant(2)


def project_to_admissible(Q, q_min, q_max):
    for i in range(len(Q)):
        proj = Max(q_min, Min(Q[i], q_max))
        Q[i] = project(proj, V)
    return Q

### Execute program

In [9]:
# --- Define mesh and function space ---

# Set time parameters
T = 4.0
num_steps = 100
dt = T/num_steps

# Define function space
nx = 32
ny = 32
mesh = UnitSquareMesh(nx,ny)
V = FunctionSpace(mesh, 'CG', 1)

# --- Specify parameters ---

element = FiniteElement("CG", triangle, 1)

q_min = Constant(0.0)
q_max = Constant(1.0)

a0 = 1.0
a1 = 1e-01
a_tol = 1e-14
a_exp = Expression('sqrt(pow(x[0] - 0.3, 2) + pow(x[1] - 0.4, 2)) <= 0.2 + tol ? a0 : a1', element=element, a0=a0, a1=a1, tol=a_tol)
a_const = interpolate(a_exp, V)
rho = 0.0
lambda_ = 1.4
alpha = 1e-05
r = project(Expression("0.01*sqrt(pow(x[0],2) + pow(x[1], 2))", element=element), V)

In [10]:
# --- Initialization ---

# State
u_0 = Constant(0.7)  # Expression('b*exp(-a*(pow(x[0]-0.25, 2) + pow(x[1]-0.25, 2)))', degree=2, a=10, b=20)
u_n = interpolate(u_0, V)

# Control
str1 = 'b*exp(-a*pow(x[0] - (0.5 - 0.25*sin(3.14*t)), 2) - a*pow(x[1] - (0.5 - 0.25*cos(3.14*t)), 2)) + '
str2 = 'b*exp(-a*pow(x[0] - (0.5 + 0.25*sin(3.14*t)), 2) - a*pow(x[1] - (0.5 + 0.25*cos(3.14*t)), 2))'
string = str1 + str2

q = Constant(1.5)  # Expression(string, degree=2, a=50, b=10, t=0)
# q = Expression('sqrt(pow(x[0] - 0.3, 2) + pow(x[1] - 0.4, 2)) <= 0.2 + tol ? a1 : a0', element=element, a0=a0, a1=a1, tol=a_tol)
t = 0
Q = [interpolate(q, V)]
for i in range(num_steps):
    t += dt
    q.t = t
    Q.append(interpolate(q, V))
Q = np.asarray(Q)


# Adjoint variable
p_end = Constant(-1*lambda_)
p_n = interpolate(p_end, V)

In [11]:
# --- Projected gradient loop ---
tol = 1e-02
v_k = 1
v_k_norm = 1

j_prev = 0
j_curr = 10

print("Initial objective function value: {}".format(j_curr))

it = 1
while v_k_norm > tol:
    print("Iteration no. {}".format(it))
    it += 1
    j_prev = j_curr
    
    U = modules.solve_state_eq(u_0, Q, V)
    P = modules.solve_adjoint_eq(p_end, U, Q, V)
    v_k = -1*dj(U, Q, P)
    [s_k, j_curr] = armijo(v_k, U, Q, P)
    
    Q = Q + s_k*v_k
    Q = project_to_admissible(Q, q_min, q_max, V)
    
    v_k_norm = grad_norm(v_k, V, T, num_steps)
    print("Step length found: s_k = {}".format(float(s_k)))
    print("New function value: j_curr = {}".format(j_curr))
    print("Decrease in function value: j_prev - j_curr = {}".format(j_prev - j_curr))
    print("Norm of gradient: {}".format(v_k_norm))
    print("\n")

Initial objective function value: 10
Iteration no. 1


NameError: name 'T' is not defined